# Writing SPARQL queries

The goal of this assignment is to construct and test SPARQL queries for the questions you submitted for assignment 1. 

## Assignment 2a

Formulate SPARQL queries. You can test your queries at https://query.wikidata.org  If you are happy, store the queries as Python strings as suggested below. Include the question in English as a comment 


In [60]:
# What is the scientific name for cat?
q1 = '''SELECT ?nameLabel
WHERE {
  wd:Q146 p:P31 ?statement.
  ?statement ps:P31 wd:Q55983715.
  ?statement pq:P642 ?name.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}'''

# Who was the oldest cat?
q2 = '''SELECT ?catLabel (ROUND((?death - ?birth)/365.2425) AS ?age)
WHERE {
  ?cat wdt:P31 wd:Q146 .
  ?cat wdt:P570 ?death .
  ?cat wdt:P569 ?birth .
  FILTER(isLiteral(?birth)).
  FILTER(isLiteral(?death)).
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # Helps get the label in your language, if not, then en language
}
ORDER BY desc(?age)
LIMIT 1
'''

# When did Unsinkable Sam die?
q3 = '''SELECT ?death
WHERE { wd:Q893453 wdt:P570  ?death. }
'''

# How many dog breeds are there?
q4 = '''SELECT (COUNT (DISTINCT ?breed) AS ?count)
WHERE { ?breed wdt:P31 wd:Q39367 .}
'''

# Are bananas berries?
q5 = '''ASK  {wd:Q503 wdt:P279 wd:Q13184 }'''

# What is the hottest chili pepper?
q6 = '''SELECT ?itemLabel ?scoville
WHERE {
?item wdt:P279 wd:Q165199 .
?item wdt:P2658 ?scoville
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
}
ORDER BY desc(?scoville)
LIMIT 1
'''

# How heigh is an elephant?
q7 = '''SELECT ?height ?unitHeightLabel
WHERE {  
  wd:Q7378 p:P2048 ?stmnodeHeight .
  ?stmnodeHeight       psv:P2048                   ?valuenodeHeight.
  ?valuenodeHeight     wikibase:quantityAmount     ?height.
  ?valuenodeHeight     wikibase:quantityUnit       ?unitHeight.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }}
'''

# Are sheeps herbivores?
q8 = '''ASK {wd:Q7368 wdt:P279 wd:Q59099}'''

# What is the sound of an eagle called?
q9 = '''SELECT ?nameLabel  WHERE {
  wd:Q2092297 wdt:P4733 ?name
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }}
'''

# When did we start cultivating cucumbers?
q10 = '''SELECT ?date WHERE {
  wd:Q2735883 p:P279 ?statement.
  ?statement ps:P279 wd:Q11004.
  ?statement pq:P580 ?date.
  FILTER(isLiteral(?date)).}
'''

## Assignment 2b: Querying the Wikidata SPARQL Endpoint

Write a function that sends a query to the sparql endpoint for wikidata, an which prints the answers. The function takes one argument, which is a variable whose value is (the string of) a SPARQL query, i.e. yourfunction(q1) should return the answer for your first query is that has variable q1.  

Make sure you have the requests library. The code below illustrates how you can use the requests.get function to send a query to a sparql endpoint and get results back as a json object (a python dictionary). See the slides for examples of how to access the contents of that json object and display the answers. 

Test the function on your 10 queries to see whether they all give the expected answers.

In [61]:
import requests

url = 'https://query.wikidata.org/sparql'
def queryWikidata(query) : 
    data = requests.get(url,
        params={'query': query, 'format': 'json'}).json()
    if 'ASK' in query:
        print(data['boolean'])
    else:
        for item in data['results']['bindings']:
            for var in item :
                print('{}\t{}'.format(var,item[var]['value']))

In [62]:
questions = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10]
for q in questions:
    print('-------------------------')
    queryWikidata(q)
print('-------------------------')

-------------------------
nameLabel	Felis catus
nameLabel	Felis silvestris catus
-------------------------
catLabel	Creme Puff
age	38
-------------------------
death	1955-01-01T00:00:00Z
-------------------------
count	813
-------------------------
True
-------------------------
scoville	1569300
itemLabel	Carolina Reaper
-------------------------
height	4
unitHeightLabel	metre
-------------------------
True
-------------------------
nameLabel	eagle cry
-------------------------
date	-2999-01-01T00:00:00Z
-------------------------


### Date values problem
For date answers, there doesn't seem to be an optimal way to return a formatted value just with the request. There is the following trick:
```
(CONCAT(STR(MONTH(?date)), 
                      "/", 
          STR(DAY(?date)), 
                      "/", 
         STR(YEAR(?date)))
            as ?dateLabel)
```
But it doesn't seem to work with BCE dates:
```
dateLabel: 1/1/-2999
```